In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/leaf-classification/train.csv.zip
/kaggle/input/leaf-classification/images.zip
/kaggle/input/leaf-classification/test.csv.zip
/kaggle/input/leaf-classification/sample_submission.csv.zip


# Load data


In [8]:
train_data = pd.read_csv('/kaggle/input/leaf-classification/train.csv.zip')
test_data = pd.read_csv('/kaggle/input/leaf-classification/test.csv.zip')

# Analyze data

**Describe data**

* look at columns
* check shape
* check null columns

In [9]:
train_data.describe()

,id,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
count,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,...,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000,990.000000
mean,799.595960,0.017412,0.028539,0.031988,0.023280,0.014264,0.038579,0.019202,0.001083,0.007167,...,0.036501,0.005024,0.015944,0.011586,0.016108,0.014017,0.002688,0.020291,0.008989,0.019420
std,452.477568,0.019739,0.038855,0.025847,0.028411,0.018390,0.052030,0.017511,0.002743,0.008933,...,0.063403,0.019321,0.023214,0.025040,0.015335,0.060151,0.011415,0.039040,0.013791,0.022768
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,415.250000,0.001953,0.001953,0.013672,0.005859,0.001953,0.000000,0.005859,0.000000,0.001953,...,0.000000,0.000000,0.000977,0.000000,0.004883,0.000000,0.000000,0.000000,0.000000,0.000977
50%,802.500000,0.009766,0.011719,0.025391,0.013672,0.007812,0.015625,0.015625,0.000000,0.005859,...,0.004883,0.000000,0.005859,0.000977,0.012695,0.000000,0.000000,0.003906,0.002930,0.011719
75%,1195.500000,0.025391,0.041016,0.044922,0.029297,0.017578,0.056153,0.029297,0.000000,0.007812,...,0.043701,0.000000,0.022217,0.009766,0.021484,0.000000,0.000000,0.023438,0.012695,0.029297
max,1584.000000,0.087891,0.205080,0.156250,0.169920,0.111330,0.310550,0.091797,0.031250,0.076172,...,0.429690,0.202150,0.172850,0.200200,0.106450,0.578130,0.151370,0.375980,0.086914,0.141600


In [10]:
print("Colums: ", train_data.columns.values)
print("Shape: ", train_data.shape)

Colums:  ['id' 'species' 'margin1' 'margin2' 'margin3' 'margin4' 'margin5'
 'margin6' 'margin7' 'margin8' 'margin9' 'margin10' 'margin11' 'margin12'
 'margin13' 'margin14' 'margin15' 'margin16' 'margin17' 'margin18'
 'margin19' 'margin20' 'margin21' 'margin22' 'margin23' 'margin24'
 'margin25' 'margin26' 'margin27' 'margin28' 'margin29' 'margin30'
 'margin31' 'margin32' 'margin33' 'margin34' 'margin35' 'margin36'
 'margin37' 'margin38' 'margin39' 'margin40' 'margin41' 'margin42'
 'margin43' 'margin44' 'margin45' 'margin46' 'margin47' 'margin48'
 'margin49' 'margin50' 'margin51' 'margin52' 'margin53' 'margin54'
 'margin55' 'margin56' 'margin57' 'margin58' 'margin59' 'margin60'
 'margin61' 'margin62' 'margin63' 'margin64' 'shape1' 'shape2' 'shape3'
 'shape4' 'shape5' 'shape6' 'shape7' 'shape8' 'shape9' 'shape10' 'shape11'
 'shape12' 'shape13' 'shape14' 'shape15' 'shape16' 'shape17' 'shape18'
 'shape19' 'shape20' 'shape21' 'shape22' 'shape23' 'shape24' 'shape25'
 'shape26' 'shape27' 'shap

In [11]:
print("Missing values:")
print(train_data.isnull().sum())

Missing values:
id           0
species      0
margin1      0
margin2      0
margin3      0
            ..
texture60    0
texture61    0
texture62    0
texture63    0
texture64    0
Length: 194, dtype: int64


**Cool, there are no missing values =)**

# Encode data

In [12]:
def encode(train, test):
    le = LabelEncoder().fit(train.species) 
    labels = le.transform(train.species)
    classes = list(le.classes_)                   
    test_ids = test.id                 

    train = train.drop(['species', 'id'], axis=1)  
    test = test.drop(['id'], axis=1)
    
    return train, labels, test, test_ids, classes

X, y, test_data, test_ids, classes = encode(train_data, test_data)
train_data.head(1)

,id,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,1,Acer_Opalus,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,...,0.007812,0.0,0.00293,0.00293,0.035156,0.0,0.0,0.004883,0.0,0.025391


# Prepare data

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [14]:
from sklearn.preprocessing import normalize
X_train_norm=normalize(X_train)
X_test_norm=normalize(X_test)

# LinearDiscriminantAnalysis

In [15]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
clf = LinearDiscriminantAnalysis()

clf.fit(X_train_norm, y_train)
print("RF Accuracy: " + repr(round(clf.score(X_test_norm, y_test) * 100, 2)) + "%")

RF Accuracy: 97.47%


# RandomForestClassifier

In [16]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(criterion='entropy',
                             n_estimators=700,
                             min_samples_split=5,
                             min_samples_leaf=1,
                             max_features = "auto",
                             oob_score=True,
                             random_state=0,
                             n_jobs=-1)

clf.fit(X_train_norm, y_train)
print("RF Accuracy: " + repr(round(clf.score(X_test_norm, y_test) * 100, 2)) + "%")

RF Accuracy: 93.43%


# Export

In [17]:
clf = LinearDiscriminantAnalysis()
clf.fit(X_train_norm, y_train)
result = clf.predict_proba(normalize(test_data))
df = pd.DataFrame(result, columns=classes)
df.insert(0, 'id', test_ids)
df.reset_index()

print(result.shape)
filename = 'Prediction.csv'
df.to_csv(filename,index=False)
print('Saved file: ' + filename)

(594, 99)
Saved file: Prediction.csv
